In [4]:
cd ..

c:\Users\Lenovo\Desktop\Tourist Attraction Recommendation System


c:\Users\Lenovo\Desktop\Tourist Attraction Recommendation System\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [36]:
import pandas as pd 
import numpy as np 
from sklearn.metrics import mean_squared_error
import implicit  # Import the implicit library
import scipy


In [6]:
df = pd.read_csv("Dataset/reviews_output.csv")
df.head()

,Place,Reviewer Name,Reviewer Link,Reviewer Image,Rating,Date,Text,Photos,Likes Count
0,Bhaktapur Durbar Square,Yuval Oz,https://www.google.com/maps/contrib/1020865195...,https://lh3.googleusercontent.com/a/ACg8ocLJPu...,5,3 weeks ago,This is an ancient area of ​​the royal palace....,https://lh5.googleusercontent.com/p/AF1QipOSO8...,0
1,Bhaktapur Durbar Square,Adam Lloyd,https://www.google.com/maps/contrib/1112437890...,https://lh3.googleusercontent.com/a-/ALV-UjU1i...,4,a month ago,Bhaktapur lives up to the hype with regards to...,https://lh5.googleusercontent.com/p/AF1QipMKiu...,1
2,Bhaktapur Durbar Square,Ahana Sengupta,https://www.google.com/maps/contrib/1169951575...,https://lh3.googleusercontent.com/a/ACg8ocJO7r...,5,a month ago,Feels like a travel back in time with traditio...,https://lh5.googleusercontent.com/p/AF1QipOaze...,0
3,Bhaktapur Durbar Square,Biswajit Saha,https://www.google.com/maps/contrib/1125207243...,https://lh3.googleusercontent.com/a-/ALV-UjVfH...,5,2 months ago,"Among the 3 durbar squares in Kathmandu, this ...",https://lh5.googleusercontent.com/p/AF1QipPCmH...,1
4,Bhaktapur Durbar Square,Abhijat Poudel,https://www.google.com/maps/contrib/1006483766...,https://lh3.googleusercontent.com/a/ACg8ocK9qV...,5,2 months ago,A renowned world heritage site with a long his...,https://lh5.googleusercontent.com/p/AF1QipPaB9...,3


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Place           240 non-null    object
 1   Reviewer Name   240 non-null    object
 2   Reviewer Link   240 non-null    object
 3   Reviewer Image  240 non-null    object
 4   Rating          240 non-null    int64 
 5   Date            232 non-null    object
 6   Text            237 non-null    object
 7   Photos          222 non-null    object
 8   Likes Count     240 non-null    int64 
dtypes: int64(2), object(7)
memory usage: 17.0+ KB


In [21]:
df["Rating"].unique()

array([5, 4, 2, 3, 0])

In [55]:
# Prepare the user-item matrix
def prepare_user_item_matrix(data):
    user_item_matrix = data.pivot_table(
        index="Reviewer Name", columns="Place", values="Rating"
    )
    return user_item_matrix

user_item_matrix = prepare_user_item_matrix(df)

In [40]:
# Convert the user-item matrix to a sparse matrix format
user_item_matrix_sparse = user_item_matrix.fillna(0).values
user_item_matrix_sparse = scipy.sparse.csr_matrix(user_item_matrix_sparse)
user_item_matrix_sparse

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 224 stored elements and shape (188, 29)>

In [66]:
# Train-test split (80% training, 20% testing)
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Prepare user-item matrices for training and testing
train_user_item_matrix = prepare_user_item_matrix(train_data)
test_user_item_matrix = prepare_user_item_matrix(test_data)

In [67]:
# Convert to sparse matrix format
train_user_item_matrix_sparse = scipy.sparse.csr_matrix(train_user_item_matrix.fillna(0).values)
test_user_item_matrix_sparse = scipy.sparse.csr_matrix(test_user_item_matrix.fillna(0).values)


In [68]:
# Initialize and fit the ALS model
model = implicit.als.AlternatingLeastSquares(factors=50, regularization=0.1, iterations=50)
model.fit(train_user_item_matrix_sparse)

100%|██████████| 50/50 [00:00<00:00, 886.34it/s]


In [69]:

# Create a mapping for users and items
user_idx_map = {user: idx for idx, user in enumerate(train_user_item_matrix.index)}
item_idx_map = {item: idx for idx, item in enumerate(train_user_item_matrix.columns)}

In [70]:
# Function to calculate RMSE
def calculate_rmse(original_matrix, predicted_matrix):
    # Ensure both matrices have the same shape
    if original_matrix.shape != predicted_matrix.shape:
        raise ValueError("Original and predicted matrices must have the same shape.")
    
    # Create a mask for actual ratings (non-zero entries)
    mask = original_matrix != 0
    
    # Flatten both matrices to 1D arrays
    original_flat = original_matrix[mask]
    predicted_flat = predicted_matrix[mask]
    
    # Compute MSE and RMSE
    mse = mean_squared_error(original_flat, predicted_flat)
    rmse = np.sqrt(mse)
    return rmse

In [73]:
# Function to calculate RMSE
def calculate_rmse(original_matrix, predicted_matrix):
    # Ensure both matrices have the same shape
    if original_matrix.shape != predicted_matrix.shape:
        raise ValueError("Original and predicted matrices must have the same shape.")
    
    # Create a mask for actual ratings (non-zero entries)
    mask = original_matrix != 0
    
    # Flatten both matrices to 1D arrays, only keeping the non-zero entries (non-NaN)
    original_flat = original_matrix[mask]
    predicted_flat = predicted_matrix[mask]
    
    # Remove NaN values in both arrays
    valid_indices = ~np.isnan(original_flat) & ~np.isnan(predicted_flat)
    original_flat = original_flat[valid_indices]
    predicted_flat = predicted_flat[valid_indices]
    
    # Compute MSE and RMSE
    mse = mean_squared_error(original_flat, predicted_flat)
    rmse = np.sqrt(mse)
    return rmse


In [74]:
# Calculate RMSE for the valid test data
rmse = calculate_rmse(
    test_user_item_matrix.loc[valid_test_users, valid_test_items].values, 
    predicted_ratings_test_matrix
)
print(f"Root Mean Squared Error (RMSE) on Test Data: {rmse}")

Root Mean Squared Error (RMSE) on Test Data: 4.534809585149154


In [81]:
def recommend_places(user_name, num_recommendations=5):
    if user_name not in user_item_matrix.index:
        print("User not found!")
        return []
    
    # Get the index of the user in the user-item matrix
    user_idx = user_item_matrix.index.get_loc(user_name)  # This returns the index of the user

    # Check if the user index is within bounds of the ALS model's user factors
    if user_idx >= len(model.user_factors):
        print("User index out of bounds for the ALS model.")
        return []
    
    # Recommend places using the ALS model
    recommended_items = model.recommend(user_idx, user_item_matrix_sparse[user_idx], N=num_recommendations)
    
    # Extract item indices and scores from the recommended items
    item_indices, scores = recommended_items
    
    # Format the recommendations as a list of (item_name, score)
    recommendations = [(user_item_matrix.columns[int(item_idx)], score) for item_idx, score in zip(item_indices, scores)]
    
    return recommendations


In [84]:
# Step 1: List all users in the user-item matrix
print("Users in the user-item matrix:", user_item_matrix.index.tolist())

# Step 2: Recreate the user-item matrix
user_item_matrix = prepare_user_item_matrix(df)  # Recreate user-item matrix

# Step 3: Check the index for 'Yuval Oz'
valid_user_name = "1 Min Beats"
try:
    user_idx_in_matrix = user_item_matrix.index.get_loc(valid_user_name)  # Get index for Yuval Oz
    print(f"Index for '{valid_user_name}' in user-item matrix:", user_idx_in_matrix)
    recommendations = recommend_places(valid_user_name)
    print(f"Top recommendations for {valid_user_name}:\n", recommendations)
except KeyError:
    print(f"User '{valid_user_name}' not found in user-item matrix.")


Users in the user-item matrix: ['1 Min Beats', 'A A', 'Aashutosh Shiwakoti', 'Abhigyan Basak', 'Abhijat Poudel', 'Abhimanue Budhathoki', 'Abhishek Kumar Ambar', 'Abhishek Poudel', 'Abinash singh', 'Ace Rai (aaceees)', 'Adam Lloyd', 'Addy Kapur', 'Aditya Lama Lopchan', 'Ahana Sengupta', 'Akash Dhami', 'Akash Kumar', 'Akash Prajapati', 'Akash Ranjan', 'Akriti Shrestha', 'Amartya Ghosal', 'Amit Maskara', 'Ananya R (Passion Tracker)', 'Andrew', 'Anish Bhattarai', 'Anish Tamang', 'Anitha Revankar', 'Anjali raj', 'Ankit Bhatt', 'Ankush K', 'Anmol Bade', 'Anuj yadav', 'Anurag Jassal', 'Aravind S', 'Archana Kakade', 'Archana Lama', 'Aruna Karki', 'Avishek Shrestha', 'Aviv Adhikari', 'Ayush Bhattarai', 'Babak afshar ebrahimi', 'Bibek Yadav', 'Bidhan shrestha', 'Bikesh Youngya', 'Bimochan Chaudhary', 'Binesh Maharjan', 'Binuwick', 'Bipul Thapa Magar', 'Bishal Humagain', 'Biswajit Saha', 'Blossom Expeditions', 'Bob Manthy', 'CLASH', 'Carlo R. Carranza', 'Chaitanya Hegde', 'Charu Kalsy', 'Chloe Da

In [88]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Function to calculate RMSE
def calculate_rmse(actual_ratings, predicted_ratings):
    # Flatten the matrices to 1D arrays
    actual_ratings_flat = actual_ratings.flatten()
    predicted_ratings_flat = predicted_ratings.flatten()
    
    # Compute RMSE
    mse = mean_squared_error(actual_ratings_flat, predicted_ratings_flat)
    rmse = np.sqrt(mse)
    return rmse

# Function to calculate MAE
def calculate_mae(actual_ratings, predicted_ratings):
    # Flatten the matrices to 1D arrays
    actual_ratings_flat = actual_ratings.flatten()
    predicted_ratings_flat = predicted_ratings.flatten()
    
    # Compute MAE
    mae = mean_absolute_error(actual_ratings_flat, predicted_ratings_flat)
    return mae

# Example usage:
# Assuming you have a set of actual and predicted ratings matrices for the test data

# Actual ratings for the test data (you need to have the test data for evaluation)
# Ensure `valid_test_users` and `valid_test_items` are aligned with the test set
actual_ratings_test = test_user_item_matrix.loc[valid_test_users, valid_test_items].values  # Replace with your actual ratings matrix

# Initialize an empty list to store predicted ratings for test items
predicted_ratings_list = np.zeros_like(actual_ratings_test)  # Initialize an empty array to store predicted ratings

# Iterate over the users in the test data
for user_idx in range(len(valid_test_users)):
    # Extract the recommended items for the current user
    recommended_items = model.recommend(user_idx, user_item_matrix_sparse[user_idx], N=len(valid_test_items))
    item_indices, predicted_scores = recommended_items

    # Ensure predicted ratings are aligned with valid_test_items
    for i, (item_idx, score) in enumerate(zip(item_indices, predicted_scores)):
        predicted_ratings_list[user_idx, i] = score  # Update the predicted rating for the user-item pair

# Check for NaN values before calculation
if np.any(np.isnan(actual_ratings_test)) or np.any(np.isnan(predicted_ratings_list)):
    actual_ratings_test = np.nan_to_num(actual_ratings_test, nan=0)
    predicted_ratings_list = np.nan_to_num(predicted_ratings_list, nan=0)

# Calculate RMSE
rmse = calculate_rmse(actual_ratings_test, predicted_ratings_list)
print(f"Root Mean Squared Error (RMSE) on Test Data: {rmse}")

# Calculate MAE
mae = calculate_mae(actual_ratings_test, predicted_ratings_list)
print(f"Mean Absolute Error (MAE) on Test Data: {mae}")


Root Mean Squared Error (RMSE) on Test Data: 1.013412309779687
Mean Absolute Error (MAE) on Test Data: 0.24459746017664594


In [90]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# Function to calculate RMSE
def calculate_rmse(actual_ratings, predicted_ratings):
    actual_ratings_flat = actual_ratings.flatten()
    predicted_ratings_flat = predicted_ratings.flatten()
    
    # Handle NaN values by removing them
    mask = ~np.isnan(actual_ratings_flat) & ~np.isnan(predicted_ratings_flat)
    actual_ratings_flat = actual_ratings_flat[mask]
    predicted_ratings_flat = predicted_ratings_flat[mask]
    
    mse = mean_squared_error(actual_ratings_flat, predicted_ratings_flat)
    rmse = np.sqrt(mse)
    return rmse

# Function to calculate MAE
def calculate_mae(actual_ratings, predicted_ratings):
    actual_ratings_flat = actual_ratings.flatten()
    predicted_ratings_flat = predicted_ratings.flatten()
    
    # Handle NaN values by removing them
    mask = ~np.isnan(actual_ratings_flat) & ~np.isnan(predicted_ratings_flat)
    actual_ratings_flat = actual_ratings_flat[mask]
    predicted_ratings_flat = predicted_ratings_flat[mask]
    
    mae = mean_absolute_error(actual_ratings_flat, predicted_ratings_flat)
    return mae

# Function to calculate R² score
def calculate_r2(actual_ratings, predicted_ratings):
    actual_ratings_flat = actual_ratings.flatten()
    predicted_ratings_flat = predicted_ratings.flatten()
    
    # Handle NaN values by removing them
    mask = ~np.isnan(actual_ratings_flat) & ~np.isnan(predicted_ratings_flat)
    actual_ratings_flat = actual_ratings_flat[mask]
    predicted_ratings_flat = predicted_ratings_flat[mask]
    
    r2 = r2_score(actual_ratings_flat, predicted_ratings_flat)
    return r2

# Function to calculate Precision, Recall, F1 score
def calculate_precision_recall_f1(actual_ratings, predicted_ratings, threshold=3.0):
    # Convert ratings to binary values: 1 if rating >= threshold, else 0
    actual_binary = (actual_ratings >= threshold).flatten()
    predicted_binary = (predicted_ratings >= threshold).flatten()
    
    # Handle NaN values by removing them
    mask = ~np.isnan(actual_binary) & ~np.isnan(predicted_binary)
    actual_binary = actual_binary[mask]
    predicted_binary = predicted_binary[mask]
    
    precision = precision_score(actual_binary, predicted_binary)
    recall = recall_score(actual_binary, predicted_binary)
    f1 = f1_score(actual_binary, predicted_binary)
    
    return precision, recall, f1

# Example usage:
# Assuming you have a set of actual and predicted ratings matrices for the test data

# Actual ratings for the test data (you need to have the test data for evaluation)
actual_ratings_test = test_user_item_matrix.loc[valid_test_users, valid_test_items].values  # Replace with your actual ratings matrix

# Initialize an empty list to store predicted ratings for test items
predicted_ratings_list = []

# Iterate over the users in the test data
for user_idx in range(len(valid_test_users)):
    # Extract the recommended items for the current user
    recommended_items = model.recommend(user_idx, user_item_matrix_sparse[user_idx], N=len(valid_test_items))
    item_indices, predicted_scores = recommended_items

    # Map the item indices to actual item names
    recommended_items_with_scores = [
        (user_item_matrix.columns[int(item_idx)], score) for item_idx, score in zip(item_indices, predicted_scores)
    ]
    
    # Store the predicted ratings for each user-item pair (simplified for this example)
    for item, score in recommended_items_with_scores:
        predicted_ratings_list.append(score)

# Convert the list of predicted ratings into a numpy array
predicted_ratings_test_matrix = np.array(predicted_ratings_list).reshape(actual_ratings_test.shape)

# Calculate RMSE
rmse = calculate_rmse(actual_ratings_test, predicted_ratings_test_matrix)
print(f"Root Mean Squared Error (RMSE) on Test Data: {rmse}")

# Calculate MAE
mae = calculate_mae(actual_ratings_test, predicted_ratings_test_matrix)
print(f"Mean Absolute Error (MAE) on Test Data: {mae}")

# Calculate R² score
r2 = calculate_r2(actual_ratings_test, predicted_ratings_test_matrix)
print(f"R² Score on Test Data: {r2}")

# Calculate Precision, Recall, F1 score (using a threshold of 3.0 for relevance)
precision, recall, f1 = calculate_precision_recall_f1(actual_ratings_test, predicted_ratings_test_matrix)
print(f"Precision on Test Data: {precision}")
print(f"Recall on Test Data: {recall}")
print(f"F1 Score on Test Data: {f1}")


Root Mean Squared Error (RMSE) on Test Data: 4.846719755193043
Mean Absolute Error (MAE) on Test Data: 4.832396604275952
R² Score on Test Data: -168.1329851747253
Precision on Test Data: 0.0
Recall on Test Data: 0.0
F1 Score on Test Data: 0.0


c:\Users\Lenovo\Desktop\Tourist Attraction Recommendation System\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
